In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

In [4]:
train_data = datasets.MNIST("../data", train=True, download=True, transform=trans)

In [5]:
test_data = datasets.MNIST('../data', train=False, download=True, transform=trans)

In [6]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1000, shuffle=False)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        #----1----#
        x = self.conv2(x)
        x = self.conv2_drop(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        #----2----#
        x = x.view(-1, 320)
        x = self.fc1(x)
        x = F.relu(x)
        #----3----#
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #----4----#
        res = F.log_softmax(x, dim=1)
        return res

In [8]:
model = Net().to(device)

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.5)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=[0.5, 0.999])

In [10]:
def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target) # The negative log likelihood loss => nll_loss()
        loss.backward()
        optimizer.step()
        if (batch_idx + 1) % int(len(train_loader.dataset) / len(data) / 10) == 0:
            print('Train Epoch: {:2d} [{:5d}/{} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch, 
                (batch_idx + 1) * len(data), 
                len(train_loader.dataset),
                100 * (batch_idx + 1) / len(train_loader), 
                loss.item()))

In [11]:
def test(epoch):
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.max(1)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        
        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

In [12]:
for epoch in range(1, 11):
    train(epoch)
    test(epoch)

Train Epoch:  1 [ 6000/60000 ( 10%)]	Loss: 1.186374
Train Epoch:  1 [12000/60000 ( 20%)]	Loss: 0.642611
Train Epoch:  1 [18000/60000 ( 30%)]	Loss: 0.508379
Train Epoch:  1 [24000/60000 ( 40%)]	Loss: 0.582222
Train Epoch:  1 [30000/60000 ( 50%)]	Loss: 0.390512
Train Epoch:  1 [36000/60000 ( 60%)]	Loss: 0.366878
Train Epoch:  1 [42000/60000 ( 70%)]	Loss: 0.378603
Train Epoch:  1 [48000/60000 ( 80%)]	Loss: 0.276264
Train Epoch:  1 [54000/60000 ( 90%)]	Loss: 0.370660
Train Epoch:  1 [60000/60000 (100%)]	Loss: 0.504002

Test set: Average loss: 0.2752, Accuracy: 9152/10000 (92%)

Train Epoch:  2 [ 6000/60000 ( 10%)]	Loss: 0.179811
Train Epoch:  2 [12000/60000 ( 20%)]	Loss: 0.199817
Train Epoch:  2 [18000/60000 ( 30%)]	Loss: 0.289343
Train Epoch:  2 [24000/60000 ( 40%)]	Loss: 0.495287
Train Epoch:  2 [30000/60000 ( 50%)]	Loss: 0.211445
Train Epoch:  2 [36000/60000 ( 60%)]	Loss: 0.192874
Train Epoch:  2 [42000/60000 ( 70%)]	Loss: 0.289186
Train Epoch:  2 [48000/60000 ( 80%)]	Loss: 0.258984
Tra